# Create MODFLOW grid-based GeoTiff file

This notebook creates a GeoTiff raster file in which the pixels correspond to model grid cells. Rotated grids are allowed; however, at this time, cells must be square.  This requirement could be relaxed in the future, but rasters usually are composed of  square pixels in most GIS software.  Although MODFLOW grids won't allow it, the method can be used for skewed pixels as well. 

The user needs to have a polygon shapefile of the model boundary (rectangular).  The shapefile can contain multiple polygons that together define  the model grid outline.  The projection of the model grid is read from the shapefile .prj file. With a little coding, the projection could also be supplied as an EPSG code. 

The pixels are coded to take the value of the ibound array in the layer specified in the variable `ib2use`. This could be changed to take the value of any model quantity. 

In [ ]:
__author__ = 'Jeff Starn'   
%matplotlib notebook
from ipywidgets import interact, Dropdown
from IPython.display import display

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gp
import gdal
gdal.UseExceptions()

import ogr
import osr
import flopy as fp

The next cell contains user-supplied information. The `homes` variable is a list of directories that contain one or more MODFLOW name files. The directories in this list will be scanned and a list of MODFLOW files with their paths will be created. The user can select from this list in a subsequent cell.

In [ ]:
homes = ['../Models']

mfpth = '../executables/MODFLOW-NWT_1.0.9/bin/MODFLOW-NWT_64.exe'

# give the base name (no file extension) of the model grid shapefile
model_outline = 'fzmg_model_outline'
model_outline = 'SIR2016_5076'

ib2use = 0

Scan the directories in `home` looking for name files.

In [ ]:
dir_list = []
mod_list = []
i = 0

for home in homes:
    if os.path.exists(home):
        for dirpath, dirnames, filenames in os.walk(home):
            for f in filenames:
                if os.path.splitext(f)[-1] == '.nam':
                    mod = os.path.splitext(f)[0]
                    mod_list.append(mod)
                    dir_list.append(os.path.join(dirpath, f))
                    i += 1
print('    {} models read'.format(i))

Choose a name file from this list.

In [ ]:
model_area = Dropdown(
    options=mod_list,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

Make path names etc. from the selected model.

In [ ]:
model = model_area.value
nam_path = [item for item in dir_list if model in item][0]
nam_file = os.path.basename(nam_path)
model_ws = os.path.dirname(nam_path)

new_ws = os.path.join(model_ws, 'WEL')
geo_ws = os.path.dirname(model_ws)

print("working model is {}".format(nam_path))

In [ ]:
# the following information can be input directly or read from flopy

# NACP model
# delr = delc = 5280
# nrow = 250
# ncol = 500

# Fall Zone model
# delr = delc = 1056
# nrow = 750
# ncol = 250

Read the model using FLOPY. Only the BAS and DIS packages need to read to create a basic GeoTiff.

In [ ]:
print ('Reading model information')

fpmg = fp.modflow.Modflow.load(nam_file, model_ws=model_ws, exe_name=mfpth, version='mfnwt', 
                               load_only=['DIS', 'BAS6'], check=False)

dis = fpmg.get_package('DIS')
bas = fpmg.get_package('BAS6')

delr = dis.delr
delc = dis.delc
nlay = dis.nlay
nrow = dis.nrow
ncol = dis.ncol

hnoflo = bas.hnoflo
ibound = np.asarray(bas.ibound.get_value())

print ('   ... done') 

Fucntions used in the notebook

In [ ]:
def get_minmax(g):
    '''This function extracts x and y values from a polygon
    and finds the coordinate pairs at extreme values.
    
    g : Shapely Polygon or MultiPolygon object
    
    returns: array of (x, y) pairs at extreme values'''
    
    x, y = np.array(list(zip(*g.boundary.coords[:])))
    return find_minmax(x, y)

def find_minmax(x, y):
    '''This function finds the pairs of coordinates at each extreme value.
    
    x, y : array of single coordinates, x and y
    
    returns: array of (x, y) pairs at extreme values'''

    ximin = np.argmin(x)
    ximax = np.argmax(x)
    yimin = np.argmin(y)
    yimax = np.argmax(y)
    
    return np.array(((x[ximin], y[ximin]),
           (x[yimax], y[yimax]),
           (x[ximax], y[ximax]),
           (x[yimin], y[yimin])))

### Find the corner points of an arbitrary rectangular shapefile (i.e., MODFLOW grid)

Read the shapefile

In [ ]:
src = os.path.join(geo_ws, model_outline)
basin = gp.read_file(src + '.shp')

Read the shapefile's projection file (`.prj`). Convert to other formats.  The SRS object provides methods for other formats.

In [ ]:
# Read the projection associated with the shapefile (in ESRI WKT format).
with open(src +  '.prj', 'r') as f:
    prj = f.readlines()
    
# Convert the projection to Proj.4 (for geopandas and matplotlib) and WKT 
# (for open source geotiff file)
srs = osr.SpatialReference()
srs.ImportFromESRI(prj)
prj4 = srs.ExportToProj4()
wkt = srs.ExportToWkt()

# initialize with dummy array so that new arrays of the same shape can be appended
arr = np.zeros((1, 2))

# loop through all the geometries in the source shapefile and
# append the pairs of coordinates at extreme values
for geom in basin.geometry:
    if geom.type == 'Polygon':
        arr = np.append(arr, get_minmax(geom), axis=0)
    elif geom.type == 'MultiPolygon':
        for g in geom:
            arr = np.append(arr, get_minmax(g), axis=0)
    else:
        print('unrecognized geometry type; should be Polygon or MultiPolygon')

# find the global set of coordinates at extreme values (corners)
pts = find_minmax(arr[1:, 0], arr[1:, 1])

Check for errors

In [ ]:
LX = np.unique(delr)
LY = np.unique(delc)

assert LX.shape[0]==1, "grid spacing in delr is not uniform; can't use raster"
assert LY.shape[0]==1, "grid spacing in delc is not uniform; can't use raster"
assert LX==LY, "grid cells are not square; can't use raster"

L = LX

Process the corner points to find the origin with respect to the given `nrow` and `ncol` and the angle of grid rotation in radians from the positive x axis.

In [ ]:
# Find the apex (ymax) of the grid.
ymax = np.argmax(pts[:, 1])

# Wrap (roll) the lines of the array around so that the apex is at the top of the array (first line).
pts = np.roll(pts, -ymax, axis=0)

# Add the first point to the end for calculating distances
pts = np.vstack((pts, pts[0, :]))

# Calculate the length of each side.
dc = np.diff(pts, axis=0)
hyp = np.hypot(dc[:, 0], dc[:, 1])

# angle in radians from positive x axis such that negative y values produce negative angles
da = np.arctan2(dc[:, 1], dc[:, 0])

Calculate the geotransformation coordinates for the raster

In [ ]:
# the corner coordinates always have the ncol dimension to the right of the origin 

if ncol <= nrow:
    if hyp[0] <= hyp[3]:
        origin = pts[0, :]
        theta = da[0]
    else:
        origin = pts[3, :]
        theta = da[3]
elif ncol > nrow:
    if hyp[0] < hyp[3]:
        origin = pts[3, :]
        theta = da[3]
    else:
        origin = pts[0, :]
        theta = da[0]
else:
    assert np.isclose(hyp[0], hyp[3]), 'nrow = ncol but sides are not equal length'

A = L * np.cos(theta)
B = L * np.sin(theta)
D = L * np.sin(theta)
E = L * -np.cos(theta)

gt = [origin[0], A[0], B[0], origin[1], D[0], E[0]]

In [ ]:
pts

In [ ]:
ax = basin.plot()
ax.plot(arr[:,0], arr[:,1], marker='x', linestyle='None', **{'mec':'k', 'linewidth':1.0})
ax.plot(origin[0], origin[1], marker='o', linestyle='None', **{'mec':'k', 'linewidth':1.0})

Make the raster and save as a GeoTiff file

In [ ]:
dst_file = os.path.join(geo_ws, 'model_grid.tif')

if os.path.exists(dst_file):
    os.remove(dst_file)
    
driver = gdal.GetDriverByName("GTiff")
dst = driver.Create(dst_file, ncol, nrow, 1, gdal.GDT_Float32)
dst.SetProjection(wkt)
dst.SetGeoTransform(gt)
ba = dst.GetRasterBand(1)
no = ba.SetNoDataValue(0)
ar = ba.WriteArray(ibound[ib2use, :, :])
dst = None
driver = None